# Trace Consistency Tests

This notebook tests that execution traces in Talyn are deterministic, correctly structured, and robust to invalid manipulations.


## 1. Determinism with Fixed Seed


In [ ]:
import numpy as np
np.random.seed(42)
trace1 = [np.random.normal() for _ in range(5)]
np.random.seed(42)
trace2 = [np.random.normal() for _ in range(5)]
print('Traces identical:', trace1 == trace2)


## 2. Trace Structure: Node Count, Order, Value


In [ ]:
from talyn.ppl.tiny_ppl import Trace
from talyn.distributions import Normal
trace = Trace()
mu = Normal(0, 1).sample(trace=trace)
x = Normal(mu, 1).sample(trace=trace)
print('Trace node count:', len(trace.nodes))
print('Trace order:', list(trace.nodes.keys()))
print('Trace values:', list(trace.nodes.values()))


## 3. Invalid Traces: Cycle Insertion, Improper Reuse


In [ ]:
try:
    # Simulate a cycle (should raise or be caught)
    trace.nodes['a'] = 1
    trace.nodes['b'] = 'a'
    trace.nodes['a'] = 'b'
    print('Cycle not detected!')
except Exception as e:
    print('Cycle detected:', e)


## 4. Visualize Trace DAGs for Different Seeds


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
def plot_trace(seed):
    np.random.seed(seed)
    trace = Trace()
    mu = Normal(0, 1).sample(trace=trace)
    x = Normal(mu, 1).sample(trace=trace)
    G = nx.DiGraph()
    for k, v in trace.nodes.items():
        G.add_node(k)
    G.add_edge('mu', 'x')
    nx.draw(G, with_labels=True, node_color='lightblue', node_size=1500, font_size=12)
    plt.title(f'Trace DAG (seed={seed})')
    plt.show()
plot_trace(1)
plot_trace(2)


## 5. Stress Test: Branching/Circular Graph Attempts


In [ ]:
try:
    # Attempt to create a circular dependency
    trace = Trace()
    trace.nodes['a'] = 'b'
    trace.nodes['b'] = 'a'
    print('Circular dependency not detected!')
except Exception as e:
    print('Circular dependency detected:', e)
